In [1]:
import tkinter as tk
from tkinter import filedialog
import speech_recognition as sr
import pyttsx3
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import pandas as pd
from PIL import Image, ImageTk

In [2]:
ui_configs = {
    "user_input_label_x": 160,
    "user_input_label_y": 175,
    "user_input_x": 160,
    "user_input_y": 200,
    "modal_input_label_x": 160,
    "modal_input_label_y": 250,
    "modal_input_x": 160,
    "modal_input_y": 275,
    "send_button_x": 250,
    "send_button_y": 450,
    "voice_button_x": 250,
    "voice_button_y": 500,
    "smile_face": "assets/smile-face.png",
    "sad_face": "assets/sad-face.png",
    "image_x": 270,
    "image_y": 325
}

In [3]:
df = pd.read_csv("data/tweets2.csv")

predict_dict = {0: 'Olumsuz', 1: 'Olumlu'}

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Tokenizeri kuruyoruz
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train_df["Görüş"])

# LSTM modelini yüklüyoruz
lstm_model = tf.keras.models.load_model("lstm_model.keras")

C:\Users\erenn\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [4]:
# Kullanıcının sesini dinlememizde yardımcı olan sınıf
class Speech2Text():
    def __init__(self):
        pass

    # Kullanıcı konuşmaya başladığında bu fonksiyon çalışır
    def listen_user(self):
        recognizer = sr.Recognizer()
        text = ""

        while(True):
            try:
                with sr.Microphone() as microphone:
                    # Kullanıcının konuşmaya başlarken etraftaki gürültüyü engellemek için bekler
                    recognizer.adjust_for_ambient_noise(microphone, duration=0.2)
    
                    # Kullanıcının söylediğini input olarak alır
                    user_voice = recognizer.listen(microphone)

                    text = recognizer.recognize_google(user_voice, language="tr-TR")
                    text = text.lower()
    
                    print(f"Söylenen cümle: {text}")

                    break

            except sr.RequestError as e:
                print("Could not request results; {0}".format(e))
         
            except sr.UnknownValueError:
                print("unknown error occurred")

        return text

In [5]:
# Kullanıcının önüne çıkacak olan ekran
class AppUI():
    def __init__(self):
        pass

    # Daha önceden kaydettiğimiz LSTM modelini kullanarak duygu analizi yapıyoruz
    def sentimentAnalysis(self, text):
        dl_token = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen = 200)
        pred_dl = (lstm_model.predict(dl_token) > 0.5).astype(int)[0][0]
        text = predict_dict[pred_dl]

        return text

    # Sesi kaydet butonuna basıldığında çalışacak olan fonksiyon
    def listenVoice(self, user_input, modal_input, face_label):
        speech2text = Speech2Text()
        text = speech2text.listen_user()

        # Burada input alanını konuşmamıza göre düzenliyoruz
        user_input.delete(0, tk.END)
        user_input.insert(0, text)

        # Modelden çıktı alınır
        modal_output = self.sentimentAnalysis(text)

        if(modal_output == "Olumlu"):
            img = Image.open(ui_configs["smile_face"])
            img = img.resize((100, 100))
            pic = ImageTk.PhotoImage(img)

            face_label.config(image=pic)
            face_label.image = pic

        else:
            img = Image.open(ui_configs["sad_face"])
            img = img.resize((100, 100))
            pic = ImageTk.PhotoImage(img)

            face_label.config(image=pic)
            face_label.image = pic

        modal_input.config(state=tk.NORMAL)
        modal_input.delete(0, tk.END)
        modal_input.insert(0, modal_output)
        modal_input.config(state=tk.DISABLED)

    def sendText(self, user_input, modal_input, face_label):
        # Kullanıcının girdiği inputu alıyoruz
        text = user_input.get()

        if(text != "" or text != " "):
            # Modelden çıktı alınır
            modal_output = self.sentimentAnalysis(text)

            if(modal_output == "Olumlu"):
                img = Image.open(ui_configs["smile_face"])
                img = img.resize((100, 100))
                pic = ImageTk.PhotoImage(img)
    
                face_label.config(image=pic)
                face_label.image = pic

            else:
                img = Image.open(ui_configs["sad_face"])
                img = img.resize((100, 100))
                pic = ImageTk.PhotoImage(img)
    
                face_label.config(image=pic)
                face_label.image = pic
    
            modal_input.config(state=tk.NORMAL)
            modal_input.delete(0, tk.END)
            modal_input.insert(0, modal_output)
            modal_input.config(state=tk.DISABLED)

    # Kullanıcıya ekranı göstermek için çalışır
    def show(self):
        frame = tk.Tk()

        # Pencerenin boyutu ve konumu
        frame.geometry("600x600+610+200")

        # Pencerenin başlığı
        frame.title("Duygu Analizi")

        # Kullanıcının girdiği inputun etiketi
        user_input_label = tk.Label(frame, text="Girilen Cümle", font=("Times", 12, "bold"))
        user_input_label.place(x=ui_configs["user_input_label_x"], y=ui_configs["user_input_label_y"])

        # Kullanıcının girdiği inputu
        user_input = tk.Entry(frame, width=40, font=("Times", 12, "normal"))
        user_input.place(x=ui_configs["user_input_x"], y=ui_configs["user_input_y"])

        # Modelin çıktısının etiketi
        modal_input_label = tk.Label(frame, text="Modelin Çıktısı", font=("Times", 12, "bold"))
        modal_input_label.place(x=ui_configs["modal_input_label_x"], y=ui_configs["modal_input_label_y"])

        # Modelin çıktısı
        modal_input = tk.Entry(frame, width=40, font=("Times", 12, "normal"), state=tk.DISABLED)
        modal_input.place(x=ui_configs["modal_input_x"], y=ui_configs["modal_input_y"])

        img = Image.open(ui_configs["sad_face"])
        img = img.resize((100, 100))
        pic = ImageTk.PhotoImage(img)
        
        face_label = tk.Label(frame)
        face_label.place(x=ui_configs["image_x"], y=ui_configs["image_y"])

        # Sesi kaydet butonu
        voice_button = tk.Button(frame, width=15, text="Sesi Kaydet", bg="blue", fg="white", activebackground="white", activeforeground="blue", font=24, cursor="hand2", command=lambda: self.listenVoice(user_input, modal_input, face_label))
        voice_button.place(x=ui_configs["voice_button_x"], y=ui_configs["voice_button_y"])

        send_button = tk.Button(frame, width=15, text="Gönder", bg="blue", fg="white", activebackground="white", activeforeground="blue", font=24, cursor="hand2", command=lambda: self.sendText(user_input, modal_input, face_label))
        send_button.place(x=ui_configs["send_button_x"], y=ui_configs["send_button_y"])
        
        # Pencerenin kapatılana kadar açık kalmasını sağlar
        frame.mainloop()

In [6]:
ui = AppUI()
ui.show()

Söylenen cümle: mağazaya girdiğimde i̇çerisi çok kötü kokuyordu bu yüzden hiç memnun değilim
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step
Söylenen cümle: sesinin çok güzel olduğunu duydum bayıldım
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
